In [1]:
# from neo4j import GraphDatabase

# # URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
# URI = "neo4j://localhost"
# AUTH = ("<Username>", "<Password>")

# with GraphDatabase.driver(URI, auth=AUTH) as driver:
#     driver.verify_connectivity()

# #----

# summary = driver.execute_query(
#     "CREATE (:Person {name: $name, age: $age})",
#     name="Alice",
#     age=42,
#     database_="neo4j",
# ).summary
# print("Created {nodes_created} nodes in {time} ms.".format(
#     nodes_created=summary.counters.nodes_created,
#     time=summary.result_available_after
# ))

# #----

# # Get the name of all 42 year-olds
# records, summary, keys = driver.execute_query(
#     "MATCH (p:Person {age: $age}) RETURN p.name AS name",
#     age=42,
#     database_="neo4j",
# )

# # Loop through results and do something with them
# for person in records:
#     print(person)

# # Summary information
# print("The query `{query}` returned {records_count} records in {time} ms.".format(
#     query=summary.query, records_count=len(records),
#     time=summary.result_available_after,
# ))

In [7]:
# Common data processing
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Connect to local Neo4J database

NEO4J_URI = "neo4j://localhost"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "admin123"
NEO4J_DATABASE = "neo4j"

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [5]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
loader = PDFPlumberLoader("ISBANK2023.pdf")
docs = loader.load()
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)
# Check the number of pages
print("Number of pages in the PDF:",len(docs))

# Load the random page content
docs[2].page_content

/var/folders/fx/cz_1x_6x0x99jmwy2v17jx540000gn/T/ipykernel_17065/3376357376.py:6: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
/Users/murathansaygili/Desktop/knowledge-graph/venv_graph/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Number of pages in the PDF: 237


'An Overview Of İşbank Looking Into the Future How We Create Value Reliable Financial Actor Responsible Operations Good Corporate Citizen Financial Reports and Annexes\nIntroduction\nThe Bank of Türkiye and the Future\nfor Exactly 100 Years!\nCelebrating its century- While celebrating our 100th anniversary, we are reinforcing highlighting not just our\naccomplishments but also our long-standing dedication to responsible banking that has\nold corporate heritage by\nshaped our identity. This report summarizes our efforts towards a sustainable future and is a\nblending it with innovation testament to our commitment to environmental, social, and governance responsibilities.\nand a vision of the future,\nCelebrating its 100th anniversary in 2024 as one of our Republic\'s longest-established\nİşbank proudly presents institutions, İşbank is an institution synonymous with trust and reputation. Since its\nfoundation, İşbank has been committed to supporting the country\'s economy. As a\nits 100t

In [3]:
# Create a sample dataframe
import pandas as pd

# Data for the dataframe
data = {
    'Character': [
        'Luke Skywalker', 'Darth Vader (Anakin)', 'Princess Leia Organa', 'Han Solo', 'Yoda',
        'Obi-Wan Kenobi', 'Palpatine (Emperor)', 'Chewbacca', 'R2-D2', 'C-3PO',
        'Boba Fett', 'Jabba the Hutt', 'Lando Calrissian', 'Padmé Amidala', 'Qui-Gon Jinn',
        'Mace Windu', 'Ahsoka Tano', 'Kylo Ren (Ben Solo)', 'Rey Skywalker', 'Finn',
        'Poe Dameron', 'Count Dooku', 'Darth Maul', 'Jango Fett', 'General Grievous',
        'Rose Tico', 'Captain Phasma', 'BB-8', 'Admiral Ackbar', 'Wedge Antilles',
        'Mon Mothma', 'Ezra Bridger', 'Sabine Wren', 'Kanan Jarrus', 'Grand Admiral Thrawn',
        'Jyn Erso', 'Cassian Andor', 'Chirrut Îmwe', 'Saw Gerrera', 'Orson Krennic'
    ],
    'First Movie': [
        'Star Wars: A New Hope', 'Star Wars: A New Hope', 'Star Wars: A New Hope', 'Star Wars: A New Hope',
        'Star Wars: The Empire Strikes Back', 'Star Wars: A New Hope', 'Star Wars: The Empire Strikes Back', 
        'Star Wars: A New Hope', 'Star Wars: A New Hope', 'Star Wars: A New Hope',
        'Star Wars: The Empire Strikes Back', 'Star Wars: Return of the Jedi', 'Star Wars: The Empire Strikes Back',
        'Star Wars: The Phantom Menace', 'Star Wars: The Phantom Menace', 'Star Wars: The Phantom Menace',
        'Star Wars: The Clone Wars', 'Star Wars: The Force Awakens', 'Star Wars: The Force Awakens', 'Star Wars: The Force Awakens','Star Wars: The Force Awakens',
        'Star Wars: Attack of the Clones', 'Star Wars: The Phantom Menace', 'Star Wars: Attack of the Clones', 
        'Star Wars: Revenge of the Sith', 'Star Wars: The Last Jedi', 'Star Wars: The Force Awakens', 
        'Star Wars: The Force Awakens', 'Star Wars: Return of the Jedi', 'Star Wars: A New Hope', 
        'Star Wars: Return of the Jedi', 'Star Wars Rebels', 'Star Wars Rebels', 'Star Wars Rebels', 
        'Star Wars Rebels', 'Rogue One: A Star Wars Story', 'Rogue One: A Star Wars Story', 'Rogue One: A Star Wars Story',
        'Star Wars: The Clone Wars', 'Rogue One: A Star Wars Story'
    ],
    'Year': [
        1977, 1977, 1977, 1977, 1980, 1977, 1980, 1977, 1977, 1977, 
        1980, 1983, 1980, 1999, 1999, 1999, 2008, 2015, 2015, 2015,
        2002, 1999, 2002, 2005, 2017, 2015, 2015, 1983, 1977, 1983, 
        2014, 2014, 2014, 2016, 2016, 2016, 2012, 2016, 2012, 2016
    ],
    'Background Summary': [
        """Luke Skywalker is one of the most iconic characters in cinematic history, originating from the classic “Star Wars” franchise. His journey from a farm boy on the remote desert planet of Tatooine to the galaxy’s greatest Jedi Knight is a tale of hope, perseverance, self-discovery, and the eternal struggle between good and evil. As the son of Anakin Skywalker (Darth Vader) and Padmé Amidala, Luke was born into a lineage that was pivotal to the fate of the galaxy. Hidden from his father after the fall of the Jedi Order and the rise of the Galactic Empire, Luke was raised by his Uncle Owen and Aunt Beru on Tatooine. Though initially unaware of his true heritage, Luke was always drawn to something greater than the life of a moisture farmer.
Luke’s journey truly begins when he encounters the droids R2-D2 and C-3PO, who carry a message from Princess Leia Organa (his twin sister, though he does not know it yet). This message leads Luke to the reclusive Jedi Master Obi-Wan Kenobi, who reveals the basics of the Force to him and provides him with his father’s lightsaber. This meeting sets Luke on a path that will not only change his life but also alter the course of the galaxy. After his uncle and aunt are killed by Imperial stormtroopers, Luke has nothing left on Tatooine and decides to join Obi-Wan on a mission to help Princess Leia and the Rebel Alliance.

Throughout the original trilogy, Luke undergoes significant growth, evolving from an eager but inexperienced young man to a wise and powerful Jedi Knight. His journey is heavily influenced by his mentors, Obi-Wan Kenobi and Yoda. In “Star Wars: A New Hope,” Luke joins forces with Princess Leia, Han Solo, and others to destroy the Empire’s massive superweapon, the Death Star. His piloting skills in the battle showcase his natural talent and connection to the Force. It is during this time that he begins to embrace the idea of becoming a Jedi, a path that will test him physically, mentally, and spiritually.

In “The Empire Strikes Back,” Luke’s training under Yoda on Dagobah is one of the most iconic sequences in Star Wars. Here, Luke learns more about the Force, but he also confronts his own fears and doubts. The revelation that Darth Vader is his father is one of the most shocking moments in film history, and it profoundly impacts Luke. He grapples with the knowledge that the man responsible for so much pain and suffering in the galaxy is also the man who fathered him. This revelation makes Luke’s journey even more complicated, as he must decide whether to follow the path of the Jedi or risk falling to the dark side, as his father did.

In “Return of the Jedi,” Luke emerges as a fully-fledged Jedi Knight. His mission to redeem his father and defeat the Emperor becomes the central focus of his story. Luke’s compassion and refusal to give up on Anakin Skywalker ultimately lead to the redemption of his father and the downfall of the Sith. By rejecting the Emperor’s temptation to turn to the dark side, Luke proves himself to be the embodiment of the Jedi ideals of hope, selflessness, and the belief in the goodness of others. His final confrontation with Darth Vader is not just a physical battle but a test of his character and faith in the light side of the Force.

After the defeat of the Empire, Luke goes on to rebuild the Jedi Order, though this process is fraught with difficulties. His journey is expanded in the sequel trilogy, where we see an older, disillusioned Luke, who has gone into self-imposed exile after a disastrous attempt to train a new generation of Jedi. The rise of Kylo Ren (Ben Solo, Luke’s nephew) and the fall of his Jedi students deeply affects Luke, leading him to question the legacy of the Jedi and his role in the galaxy’s ongoing conflict. Despite his initial reluctance, Luke returns to the fight in “The Last Jedi” by projecting his Force presence to confront Kylo Ren and inspire the Resistance. His sacrifice allows the remnants of the Resistance to escape, and his legend grows even further, solidifying his place as one of the galaxy’s greatest heroes.

Luke Skywalker’s legacy continues to influence the Star Wars universe. He is the quintessential hero, a symbol of hope and perseverance in the face of overwhelming odds. From his humble beginnings on Tatooine to his role as the redeemer of Anakin Skywalker and savior of the galaxy, Luke’s character arc is one of the most well-developed and beloved in all of science fiction and fantasy.

This version of Luke Skywalker draws upon his appearances in the original trilogy (“A New Hope,” “The Empire Strikes Back,” and “Return of the Jedi”), as well as his expanded role in the sequel trilogy (“The Force Awakens,” “The Last Jedi,” and “The Rise of Skywalker”). His story is also explored in numerous novels, comic books, and animated series that flesh out his journey to restore the Jedi Order and maintain peace in the galaxy. Through these additional materials, fans get to see Luke in various stages of his life, from a young and idealistic Jedi-in-training to a wise but burdened mentor figure who ultimately finds redemption and peace in his final moments.

Luke Skywalker’s significance to the Star Wars mythos cannot be overstated. He represents the core themes of the saga: the battle between light and dark, the importance of family and legacy, and the enduring belief that even the darkest individuals can find redemption. His story is one of hope, not just for the galaxy, but for all those who seek to overcome their inner demons and rise above adversity.

This example provides a comprehensive character background with deep insights into Luke Skywalker’s journey and significance. Each character in your DataFrame will have a similarly detailed and expanded summary that goes beyond their initial description, pulling from movies, shows, and other Star Wars media.'
""",
        """Anakin Skywalker, who later becomes Darth Vader, is one of the most complex and tragic characters in the Star Wars saga. Born as a slave on Tatooine, Anakin’s potential in the Force was recognized at a young age by Jedi Master Qui-Gon Jinn, who believed Anakin to be the Chosen One, prophesized to bring balance to the Force. Qui-Gon’s belief in Anakin was so strong that, even as he lay dying, he implored his apprentice, Obi-Wan Kenobi, to train the boy as a Jedi. This began Anakin’s journey from an innocent, yet extraordinarily talented boy, to the dark enforcer of the Galactic Empire, Darth Vader.

Anakin’s early life was filled with hardship. Raised by his mother, Shmi Skywalker, in the harsh conditions of the desert planet, he yearned for freedom and adventure. His natural instincts and connection to the Force allowed him to become a skilled podracer, a rare feat for a human, and eventually, it was his talents that brought him into contact with Qui-Gon and Obi-Wan. However, Anakin’s fear of losing those he loved—stemming from his separation from his mother—was a driving force behind many of his decisions, which would eventually lead him to fall to the dark side.

As a Jedi, Anakin was exceptionally powerful, but his impulsive nature, emotional instability, and desire for control made him a difficult student for the Jedi Council. Although he earned a place as Obi-Wan’s apprentice, Anakin frequently questioned the Jedi’s teachings, particularly their emphasis on emotional detachment. This became especially evident when he secretly married Padmé Amidala, the former Queen of Naboo, an action that was strictly forbidden by the Jedi Code.

The most significant turning point in Anakin’s life came during the Clone Wars, a galactic conflict that spanned several years. During the war, Anakin earned a reputation as a fearless warrior and a brilliant strategist, leading countless missions for the Republic. However, his experiences in the war further fed his anger, impatience, and fear of loss. His worst fears were realized when he began having visions of Padmé dying in childbirth, similar to the dreams that foreshadowed his mother’s death. Desperate to prevent this, Anakin was seduced by Emperor Palpatine (then known as Chancellor Palpatine), who promised that the dark side of the Force could save Padmé’s life.

Palpatine, who had been manipulating Anakin for years, finally convinced him to turn against the Jedi. In his new identity as Darth Vader, Anakin slaughtered many of the Jedi, including younglings, in the infamous Jedi Temple massacre. His ultimate betrayal was in his duel with Obi-Wan Kenobi on the volcanic planet of Mustafar. The battle left Anakin horrifically injured and burned, and he was left for dead by Obi-Wan. Rescued by Palpatine, Anakin was placed in the black, mechanical armor that would become synonymous with Darth Vader, a walking symbol of the Emperor’s iron grip on the galaxy.

As Darth Vader, Anakin Skywalker became the Emperor’s right-hand enforcer, leading the Empire’s forces and hunting down the remaining Jedi. He was feared across the galaxy as the epitome of darkness, brutality, and power. The Emperor’s manipulation had turned Anakin into a tool of the dark side, and he lived in anguish, believing that his actions had caused Padmé’s death. This belief, along with his physical and emotional torment, made him one of the most feared figures in the galaxy. Yet, beneath the mask and armor, a small part of the compassionate and idealistic Anakin still remained, though buried deep within the monstrous visage of Darth Vader.

Vader’s redemption came through his son, Luke Skywalker, whose existence he was unaware of for many years. When Luke confronted Vader, he saw the good that still lingered within his father. In their final confrontation aboard the second Death Star, Luke’s refusal to turn to the dark side, even under the Emperor’s manipulation, inspired Anakin to finally break free of Palpatine’s control. In one final act of redemption, Anakin killed the Emperor, sacrificing his life in the process. His final moments were spent reconciled with his son, knowing that Luke had saved him from the darkness.

Anakin Skywalker’s story is a tale of immense tragedy and ultimate redemption. His fall to the dark side represents the dangers of unchecked emotions, fear, and the desire for control. Yet his eventual redemption through the love and persistence of his son serves as a powerful reminder of the enduring struggle between light and dark, and the possibility of salvation, even for those who seem beyond redemption.

Darth Vader remains one of the most iconic villains in film history, and his transformation from Anakin Skywalker to the Dark Lord of the Sith is central to the Star Wars mythos. His character’s journey—from a young boy with dreams of becoming a Jedi, to a feared enforcer of the Empire, and finally, to a redeemed father—illustrates the complexity of human nature and the eternal conflict between good and evil. Through his story, Star Wars explores themes of power, corruption, and the redemptive power of love and hope.""",
        """
Princess Leia Organa is one of the most iconic and influential characters in the Star Wars universe, symbolizing leadership, courage, and defiance in the face of tyranny. Born as Leia Amidala Skywalker, she is the twin sister of Luke Skywalker and the daughter of Anakin Skywalker (Darth Vader) and Padmé Amidala. To protect her from the Emperor and her father, she was adopted by Senator Bail Organa of Alderaan and raised as a member of the royal family. Growing up under the tutelage of her adoptive parents, Leia developed a strong sense of duty, justice, and compassion, eventually becoming a key figure in the Rebel Alliance and later the Resistance.

Leia’s story begins in “Star Wars: A New Hope,” where she is introduced as a young leader of the Rebellion, tasked with delivering the plans to the Death Star to the Rebel forces. Despite being captured by Darth Vader and subjected to interrogation by the Empire, Leia remains resilient, refusing to give up the location of the Rebel base. Her fortitude is evident when she witnesses the destruction of her home planet, Alderaan, at the hands of the Death Star, yet still does not falter in her mission.

Throughout the original trilogy, Leia’s role is multifaceted. She is not only a diplomatic leader and strategist for the Rebellion but also an active participant in missions, showcasing her bravery and combat skills. Her romance with Han Solo adds another dimension to her character, revealing her vulnerability and capacity for deep emotional connections. Despite her personal losses, including the revelation of her true parentage and the knowledge that Darth Vader is her father, Leia never wavers in her commitment to the cause of freedom.

In “The Empire Strikes Back” and “Return of the Jedi,” Leia’s leadership becomes even more critical. She takes part in the mission to rescue Han Solo from Jabba the Hutt, disguised as the bounty hunter Boushh. Later, she plays a vital role in the final battle against the Empire on the forest moon of Endor. It is during this time that she learns from Luke that they are siblings and that Vader is their father. Despite the shocking revelation, Leia remains steadfast, focused on the mission at hand, though the knowledge undoubtedly affects her.

Leia’s story continues in the sequel trilogy, where she transitions from a leader of the Rebel Alliance to the General of the Resistance. In “The Force Awakens,” it is revealed that her son, Ben Solo (Kylo Ren), has fallen to the dark side, leading to immense personal pain and tragedy for Leia. Despite this, she remains the backbone of the Resistance, guiding its members with wisdom and determination. Her relationship with Han Solo is strained by their shared grief over Ben’s fall, but their love and respect for one another endure.

Leia’s ultimate legacy is one of perseverance, hope, and resilience. Even in the face of overwhelming odds, personal loss, and betrayal, she remains committed to the ideals of the Rebellion and the Resistance. Her role as both a warrior and a diplomat highlights her versatility and strength. Leia’s leadership is not just about military strategy but also about inspiring others to believe in a better future, even when that future seems uncertain.

In “The Last Jedi” and “The Rise of Skywalker,” Leia’s influence extends beyond her direct actions. Though her health begins to fail, her presence as a guiding force remains strong. In her final moments, she reaches out to her son, Ben, through the Force, ultimately contributing to his redemption. Leia’s sacrifice symbolizes the enduring power of love and hope, themes that are central to the Star Wars saga.

Princess Leia’s character is a beacon of strength, not just in the Star Wars universe but also in popular culture. She is a feminist icon, representing a strong, independent woman who can hold her own in both political and combat arenas. Her legacy as a leader, a mother, and a symbol of resistance against tyranny endures through the generations of fans who continue to be inspired by her courage and determination.

Leia Organa’s story arc, from the early days of the Rebellion to her leadership in the Resistance, showcases her resilience and commitment to justice. She is a character who, despite immense personal and political challenges, never loses sight of her goals. Her ability to inspire those around her, her intelligence, and her deep compassion make her one of the most beloved characters in Star Wars. Leia’s journey reflects the broader themes of the Star Wars
        """,
        'Smuggler-turned-Rebel hero, pilot of the Millennium Falcon, and ally to Luke and Leia.',
        'Ancient Jedi Master who trained countless Jedi, including Luke Skywalker.',
        'Former Jedi Knight who trained Anakin and later guided Luke in the ways of the Force.',
        'Sith Lord and Emperor of the Galactic Empire, master of Darth Vader.',
        'Wookiee warrior and co-pilot of the Millennium Falcon, fiercely loyal to Han Solo.',
        'Astromech droid who served Anakin and later Luke, played key roles in many battles.',
        'Protocol droid fluent in many languages, companion to R2-D2, built by Anakin Skywalker.',
        'Notorious bounty hunter, son/clone of Jango Fett, and nemesis to Han Solo.',
        'Crime lord based on Tatooine, known for his dealings with bounty hunters and smugglers.',
        'Gambler, smuggler, and old friend of Han Solo who became a key figure in the Rebellion.',
        'Queen of Naboo and later Senator, mother to Luke and Leia, and wife of Anakin Skywalker.',
        'Jedi Master who discovered Anakin Skywalker and believed in the prophecy of the Chosen One.',
        'High-ranking Jedi Master on the Jedi Council, known for his skill with a lightsaber.',
        'Anakin Skywalker\'s Padawan, later a key figure in the Rebellion, and a skilled Jedi.',
        'Son of Han and Leia, conflicted Force user who turned to the dark side, became a Sith.',
        'Scavenger-turned-Jedi, granddaughter of Palpatine, who ultimately restores balance.',
        'Former Stormtrooper who defected to the Resistance and fought alongside Rey and Poe.',
        'Ace pilot of the Resistance, loyal to General Leia Organa, and a key leader in the fight.',
        'Former Jedi who became a Sith Lord, leading the Separatist movement in the Clone Wars.',
        'Sith apprentice to Darth Sidious, known for his double-bladed lightsaber and vengeance.',
        'Bounty hunter and template for the clone army, father/clone of Boba Fett.',
        'Cyborg commander of the Separatist droid army, feared for his lightsaber collection.',
        'Mechanic in the Resistance, inspired by her sister\'s sacrifice, and friend to Finn.',
        'Ruthless commander of the First Order\'s stormtroopers, known for her chrome armor.',
        'Spherical droid belonging to Poe Dameron, key in the Resistance\'s mission against the First Order.',
        'Mon Calamari leader in the Rebel Alliance, known for his tactical expertise ("It\'s a trap!").',
        'Rebel Alliance pilot who fought in major battles including the Death Star and Endor.',
        'Political leader of the Rebel Alliance, instrumental in forming the Rebellion against the Empire.',
        'Orphan-turned-Jedi who fought against the Empire and helped found the Rebel Alliance.',
        'Mandalorian warrior and artist, key member of the Ghost crew fighting the Empire.',
        'Former Jedi Knight who survived Order 66, mentor to Ezra Bridger, and Rebel leader.',
        'Brilliant Imperial strategist, known for his cunning and leadership in the Empire.',
        'Rebel fighter who led the mission to steal the Death Star plans, sacrificing herself in the process.',
        'Rebel spy and captain, key figure in the mission to acquire the Death Star plans.',
        'Blind warrior-monk who believes in the Force, ally to Jyn Erso and Cassian Andor.',
        'Rebel extremist who fought against both the Separatists and the Empire.',
        'Director of the Death Star project for the Empire, rival to Tarkin and other Imperial officers.'
    ],
    'Source':["source"]*40
}

# Create the dataframe
df = pd.DataFrame(data)

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 10,
    length_function = len,
    is_separator_regex = False,
)

In [37]:
df_tmp = df.iloc[:1]

In [38]:
def split_data_from_file(df):
    chunks_with_metadata = [] # use this to accumlate chunk records

    for i in range(df.shape[0]):
        print(f'Processing {i}') 
        item_text = df["Background Summary"].iloc[i] # grab the text of the item
        item_text_chunks = text_splitter.split_text(item_text) # split the text into chunks
        chunk_seq_id = 0
        for chunk in item_text_chunks:
            form_id = "star-wars-characters"
            # finally, construct a record with metadata and the chunk text
            chunks_with_metadata.append({
                'text': chunk, 
                # metadata from looping...
                'chunkSeqId': chunk_seq_id,
                # constructed metadata...
                'formId': f'{form_id}', # pulled from the filename
                'chunkId': f'{form_id}-{i}-chunk{chunk_seq_id:04d}',
                # metadata from file...
                'character': df['Character'].iloc[i],
                'movie': df['First Movie'].iloc[i],
                'year': df['Year'].iloc[i],
                'source': df['Source'].iloc[i],
            })
            chunk_seq_id += 1
        print(f'\tSplit into {chunk_seq_id} chunks')
    return chunks_with_metadata

In [39]:
first_file_chunks = split_data_from_file(df_tmp)

Processing 0
	Split into 68 chunks


In [40]:
len(first_file_chunks)

68

In [41]:
first_file_chunks

[{'text': 'Luke Skywalker is one of the most iconic characters in cinematic history, originating from the',
  'chunkSeqId': 0,
  'formId': 'star-wars-characters',
  'chunkId': 'star-wars-characters-0-chunk0000',
  'character': 'Luke Skywalker',
  'movie': 'Star Wars: A New Hope',
  'year': 1977,
  'source': 'source'},
 {'text': 'from the classic “Star Wars” franchise. His journey from a farm boy on the remote desert planet of',
  'chunkSeqId': 1,
  'formId': 'star-wars-characters',
  'chunkId': 'star-wars-characters-0-chunk0001',
  'character': 'Luke Skywalker',
  'movie': 'Star Wars: A New Hope',
  'year': 1977,
  'source': 'source'},
 {'text': 'planet of Tatooine to the galaxy’s greatest Jedi Knight is a tale of hope, perseverance,',
  'chunkSeqId': 2,
  'formId': 'star-wars-characters',
  'chunkId': 'star-wars-characters-0-chunk0002',
  'character': 'Luke Skywalker',
  'movie': 'Star Wars: A New Hope',
  'year': 1977,
  'source': 'source'},
 {'text': 'self-discovery, and the eternal

In [42]:
cypher = """
MATCH (n)
DETACH DELETE n
"""
kg.query(cypher)

[]

In [43]:
merge_chunk_node_query = """
MERGE(mergedChunk:Chunk {chunkId: $chunkParam.chunkId})
    ON CREATE SET 
        mergedChunk.character = $chunkParam.character,
        mergedChunk.movie = $chunkParam.movie, 
        mergedChunk.year = $chunkParam.year,
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId, 
        mergedChunk.text = $chunkParam.text,
        mergedChunk.source = $chunkParam.source,
        mergedChunk.formId = $chunkParam.formId
RETURN mergedChunk
"""

In [44]:
kg.query(merge_chunk_node_query, 
         params={'chunkParam':first_file_chunks[0]})

[{'mergedChunk': {'formId': 'star-wars-characters',
   'character': 'Luke Skywalker',
   'movie': 'Star Wars: A New Hope',
   'year': 1977,
   'text': 'Luke Skywalker is one of the most iconic characters in cinematic history, originating from the',
   'source': 'source',
   'chunkId': 'star-wars-characters-0-chunk0000',
   'chunkSeqId': 0}}]

In [45]:
kg.query("""
CREATE CONSTRAINT unique_chunk IF NOT EXISTS 
    FOR (c:Chunk) REQUIRE c.chunkId IS UNIQUE
""")


[]

In [46]:
node_count = 0
for chunk in first_file_chunks:
    print(f"Creating `:Chunk` node for chunk ID {chunk['chunkId']}")
    kg.query(merge_chunk_node_query, 
            params={
                'chunkParam': chunk
            })
    node_count += 1
print(f"Created {node_count} nodes")

Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0000
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0001
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0002
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0003
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0004
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0005
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0006
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0007
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0008
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0009
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0010
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0011
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0012
Creating `:Chunk` node for chunk ID star-wars-characters-0-chunk0013
Creating `:Chunk` node for chunk I

In [47]:
kg.query("""
         MATCH (n)
         RETURN count(n) as nodeCount
         """)

[{'nodeCount': 68}]

In [48]:
import ollama

In [49]:
# kg.query("""DROP INDEX form_10k_chunks""")

In [50]:
kg.query("""
         CREATE VECTOR INDEX `form_10k_chunks` IF NOT EXISTS
          FOR (c:Chunk) ON (c.textEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 768,
            `vector.similarity_function`: 'cosine'    
         }}
""")

[]

In [51]:
def get_ollama_embedding(text):
    return ollama.embeddings(model='nomic-embed-text', prompt=text)["embedding"]

kg.query("""
    MATCH (chunk:Chunk) WHERE chunk.textEmbedding IS NULL
    WITH chunk
    CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", $vector)
    """, 
    params={"vector": get_ollama_embedding(chunk['text'])}
)

[]

In [52]:
kg.refresh_schema()
print(kg.schema)

Node properties are the following:
Chunk {character: STRING, text: STRING, source: STRING, textEmbedding: LIST, chunkId: STRING, movie: STRING, chunkSeqId: INTEGER, formId: STRING, year: INTEGER}
Relationship properties are the following:

The relationships are the following:



In [53]:
def neo4j_vector_search(question):
    """Search for similar nodes using the Neo4j vector index."""
    question_embedding = get_ollama_embedding(question)
    
    vector_search_query = """
      CALL db.index.vector.queryNodes($index_name, $top_k, $question_embedding) yield node, score
      RETURN score, node.text AS text
    """
    
    similar = kg.query(vector_search_query, 
                       params={
                        'question_embedding': question_embedding,
                        'index_name': 'form_10k_chunks', 
                        'top_k': 10})
    
    return similar

In [54]:
search_results = neo4j_vector_search(
    'In a single sentence, who is Luke Skywalker.'
)

In [55]:
search_results

[{'score': 0.7663984298706055,
  'text': 'from the classic “Star Wars” franchise. His journey from a farm boy on the remote desert planet of'},
 {'score': 0.7663984298706055,
  'text': 'planet of Tatooine to the galaxy’s greatest Jedi Knight is a tale of hope, perseverance,'},
 {'score': 0.7663984298706055,
  'text': 'self-discovery, and the eternal struggle between good and evil. As the son of Anakin Skywalker'},
 {'score': 0.7663984298706055,
  'text': 'Skywalker (Darth Vader) and Padmé Amidala, Luke was born into a lineage that was pivotal to the'},
 {'score': 0.7663984298706055,
  'text': 'to the fate of the galaxy. Hidden from his father after the fall of the Jedi Order and the rise of'},
 {'score': 0.7663984298706055,
  'text': 'rise of the Galactic Empire, Luke was raised by his Uncle Owen and Aunt Beru on Tatooine. Though'},
 {'score': 0.7663984298706055,
  'text': 'Though initially unaware of his true heritage, Luke was always drawn to something greater than the'},
 {'score': 

In [56]:
class OllamaEmbeddings:
    """Custom embeddings class for generating embeddings using the local Ollama model."""
    
    def __init__(self, model_name='nomic-embed-text'):
        self.model_name = model_name

    def embed_query(self, text):
        """Generate embeddings for a single text input using Ollama."""
        result = ollama.embeddings(model=self.model_name, prompt=text)
        
        if isinstance(result, dict) and 'embedding' in result:
            embedding = result['embedding']
            if isinstance(embedding, list) and all(isinstance(x, (float, int)) for x in embedding):
                return embedding
            else:
                raise ValueError("Ollama embedding is not a valid numerical array.")
        else:
            raise ValueError("Failed to retrieve embedding from Ollama.")

    def embed_documents(self, texts):
        """Generate embeddings for a list of documents."""
        return [self.embed(text) for text in texts]

In [57]:
# Global constants
VECTOR_INDEX_NAME = 'form_10k_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

In [58]:
ollama_embeddings = OllamaEmbeddings(model_name='nomic-embed-text')

neo4j_vector_store = Neo4jVector.from_existing_graph(
    embedding=ollama_embeddings,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    node_label=VECTOR_NODE_LABEL,
    text_node_properties=[VECTOR_SOURCE_PROPERTY],
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 67, offset: 66} for query: "UNWIND $data AS row MATCH (n:`Chunk`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'textEmbedding', row.embedding) YIELD node RETURN count(*)"


In [59]:
retriever = neo4j_vector_store.as_retriever()

In [60]:
from langchain.llms.base import LLM
from typing import List

class OllamaLLM(LLM):
    """Custom LLM class to interface with the local Ollama model."""

    model_name: str = "llama3.2:3b"

    def _call(self, prompt: str, stop: List[str] = None) -> str:
        """Generate a response from the Ollama model."""
        result = ollama.chat(model=self.model_name, messages=[{'role': 'user', 'content': prompt}], stream=False)
        
        if isinstance(result, dict) and 'message' in result and 'content' in result['message']:
            return result['message']['content']
        else:
            raise ValueError(f"Failed to retrieve response from Ollama. Result: {result}")

    @property
    def _identifying_params(self) -> dict:
        """Return the identifying parameters of the model."""
        return {"model_name": self.model_name}
    
    @property
    def _llm_type(self) -> str:
        """Return the type of the LLM as required by the LangChain framework."""
        return "ollama"

In [61]:
ollama_llm = OllamaLLM(model_name='llama3.2')

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=ollama_llm,
    chain_type="stuff", 
    retriever=retriever
)

def prettychain(question: str) -> str:
    """Pretty print the chain's response to a question"""
    response = chain({"question": question}, return_only_outputs=True)
    
    print(textwrap.fill(response['answer'], 60))
    
    # # Optionally, you can also print the sources if needed
    # if 'sources' in response:
    #     print("\nSources:")
    #     for source in response['sources']:
    #         print(source)

In [62]:
prettychain("In a single sentence, who is Luke Skywalker.")

I don't know who Luke Skywalker is.


# Create the movie node

In [203]:
cypher = """
  MATCH (anyChunk:Chunk)
  WITH anyChunk LIMIT 1
  RETURN anyChunk {.movie, .year, .formId, .source} as movieInfo
"""
movie_info_list = kg.query(cypher)

movie_info_list


[{'movieInfo': {'source': 'source',
   'formId': 'star-wars-characters',
   'movie': 'Star Wars: A New Hope',
   'year': 1977}}]

In [204]:
movie_info = movie_info_list[0]["movieInfo"]

In [205]:
movie_info

{'source': 'source',
 'formId': 'star-wars-characters',
 'movie': 'Star Wars: A New Hope',
 'year': 1977}

In [206]:
cypher = """
    MERGE (f:Form {formId: $movieInfoParam.formId })
      ON CREATE 
        SET f.names = $movieInfoParam.movie
        SET f.source = $movieInfoParam.year
        SET f.movie = $movieInfoParam.movie
        SET f.year = $movieInfoParam.year

"""

kg.query(cypher, params={'movieInfoParam': movie_info})

[]

### Create a linked list of Chunk nodes for each section
- Start by identifying chunks from the same section

In [207]:
cypher = """
  MATCH (from_same_form:Chunk)
    WHERE from_same_form.formId = $formIdParam
  RETURN from_same_form {.formId, .chunkId, .chunkSeqId} as chunkInfo
    LIMIT 10
"""

kg.query(cypher, params={'formIdParam': movie_info['formId']})

[{'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0000',
   'chunkSeqId': 0}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0001',
   'chunkSeqId': 1}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0002',
   'chunkSeqId': 2}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0003',
   'chunkSeqId': 3}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0004',
   'chunkSeqId': 4}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0005',
   'chunkSeqId': 5}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0006',
   'chunkSeqId': 6}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0007',
   'chunkSeqId': 7}},
 {'chunkInfo': {'formId'

In [208]:
cypher = """
  MATCH (from_same_form:Chunk)
    WHERE from_same_form.formId = $formIdParam
  RETURN from_same_form {.formId, .chunkId, .chunkSeqId} as chunkInfo 
    ORDER BY from_same_form.chunkSeqId ASC
    LIMIT 10
"""

kg.query(cypher, params={'formIdParam': movie_info['formId']})

[{'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0000',
   'chunkSeqId': 0}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0001',
   'chunkSeqId': 1}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0002',
   'chunkSeqId': 2}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0003',
   'chunkSeqId': 3}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0004',
   'chunkSeqId': 4}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0005',
   'chunkSeqId': 5}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0006',
   'chunkSeqId': 6}},
 {'chunkInfo': {'formId': 'star-wars-characters',
   'chunkId': 'star-wars-characters-0-chunk0007',
   'chunkSeqId': 7}},
 {'chunkInfo': {'formId'

In [209]:
cypher = """
  MATCH (from_same_section:Chunk)
  WHERE from_same_section.formId = $formIdParam
  WITH from_same_section {.formId, .chunkId, .chunkSeqId} 
    ORDER BY from_same_section.chunkSeqId ASC
    LIMIT 10
  RETURN collect(from_same_section) // NEW!!!
"""

kg.query(cypher, params={'formIdParam': movie_info['formId']})


[{'collect(from_same_section)': [{'formId': 'star-wars-characters',
    'chunkId': 'star-wars-characters-0-chunk0000',
    'chunkSeqId': 0},
   {'formId': 'star-wars-characters',
    'chunkId': 'star-wars-characters-0-chunk0001',
    'chunkSeqId': 1},
   {'formId': 'star-wars-characters',
    'chunkId': 'star-wars-characters-0-chunk0002',
    'chunkSeqId': 2},
   {'formId': 'star-wars-characters',
    'chunkId': 'star-wars-characters-0-chunk0003',
    'chunkSeqId': 3},
   {'formId': 'star-wars-characters',
    'chunkId': 'star-wars-characters-0-chunk0004',
    'chunkSeqId': 4},
   {'formId': 'star-wars-characters',
    'chunkId': 'star-wars-characters-0-chunk0005',
    'chunkSeqId': 5},
   {'formId': 'star-wars-characters',
    'chunkId': 'star-wars-characters-0-chunk0006',
    'chunkSeqId': 6},
   {'formId': 'star-wars-characters',
    'chunkId': 'star-wars-characters-0-chunk0007',
    'chunkSeqId': 7},
   {'formId': 'star-wars-characters',
    'chunkId': 'star-wars-characters-0-chunk

### Add a NEXT relationship between subsequent chunks
- Use the `apoc.nodes.link` function from Neo4j to link ordered list of `Chunk` nodes with a `NEXT` relationship
- Do this for just the "Item 1" section to start

In [210]:
cypher = """
  MATCH (from_same_section:Chunk)
  WHERE from_same_section.formId = $formIdParam
  WITH from_same_section
    ORDER BY from_same_section.chunkSeqId ASC
  WITH collect(from_same_section) as section_chunk_list
    CALL apoc.nodes.link(
        section_chunk_list, 
        "NEXT", 
        {avoidDuplicates: true}
    )  // NEW!!!
  RETURN size(section_chunk_list)
"""

kg.query(cypher, params={'formIdParam': movie_info['formId']})


[{'size(section_chunk_list)': 38}]

In [211]:
kg.refresh_schema()
print(kg.schema)

Node properties are the following:
Chunk {character: STRING, text: STRING, source: STRING, textEmbedding: LIST, chunkId: STRING, movie: STRING, chunkSeqId: INTEGER, formId: STRING, year: INTEGER},Form {names: STRING, formId: STRING, source: INTEGER, movie: STRING, year: INTEGER}
Relationship properties are the following:

The relationships are the following:
(:Chunk)-[:NEXT]->(:Chunk)


In [212]:
cypher = """
  MATCH (c:Chunk), (f:Form)
    WHERE c.formId = f.formId
  MERGE (c)-[newRelationship:PART_OF]->(f)
  RETURN count(newRelationship)
"""

kg.query(cypher)

[{'count(newRelationship)': 38}]